# Dask Delayed

Материалы:
* Макрушин С.В. Лекция 13: Dask Delayed
* https://docs.dask.org/en/latest/delayed.html
* Jesse C. Daniel. Data Science with Python and Dask.


## Задачи для совместного разбора

![](https://i.imgur.com/AwiN8y6.png)
![](https://i.imgur.com/ceY6guU.png)

1. Напишите 2 функции, имитирующие CPU-bound задачу и IO-bound задачу:

`cpu_task()`: генерирует 100 тыс. случайных чисел и возвращает их сумму (без использования `numpy`)

`io_task()`: "спит" 0.1 сек, затем генерирует случайное число и возвращает его

Замерьте время выполнения 100 последовательных вызовов каждой из этих функций. Распараллелив вычисления при помощи `dask.delayed`, сократите время выполнения. Исследуйте, как зависит время вычислений от выбранного планировщика `scheduler`.

In [1]:
import random
import time


def cpu_task():
	lst = [random.randint(0, 10) for _ in range(100_000)]
	return sum(lst)

def io_task():
	time.sleep(0.1)
	return random.randint(0, 10)

In [2]:
%%time
res = [cpu_task() for _ in range(100)]

Wall time: 4.07 s


In [3]:
%%time
res = [io_task() for _ in range(100)]

Wall time: 10.8 s


In [8]:
from dask import delayed
from dask import compute

In [5]:
cpu_task_delayed = delayed(cpu_task)

In [6]:
cpu_task_delayed

Delayed('cpu_task-ac40120a-5ea1-41a4-a72f-fcc99c234c17')

In [9]:
%%time
res = [cpu_task_delayed() for _ in range(100)]
compute(res)

Wall time: 4.11 s


([499472,
  499501,
  500654,
  499008,
  499409,
  501652,
  499998,
  500428,
  500694,
  500807,
  500337,
  500528,
  498999,
  500009,
  500968,
  500769,
  499749,
  500257,
  499002,
  500213,
  499322,
  499651,
  499006,
  499855,
  500670,
  499468,
  500032,
  500305,
  500738,
  500681,
  499722,
  499936,
  499420,
  499469,
  499147,
  498455,
  499582,
  500873,
  498704,
  499185,
  499081,
  499935,
  501697,
  499051,
  500626,
  498240,
  499822,
  500641,
  500555,
  499455,
  499789,
  500923,
  499739,
  500361,
  499573,
  500466,
  499965,
  499165,
  500741,
  498951,
  498171,
  501325,
  501753,
  500672,
  499119,
  497931,
  499771,
  500095,
  498941,
  499373,
  501018,
  500978,
  500876,
  499428,
  501467,
  500921,
  500299,
  498792,
  500449,
  502080,
  500052,
  500557,
  500347,
  498825,
  502218,
  500087,
  500189,
  499657,
  499366,
  500435,
  499825,
  500746,
  499879,
  499346,
  502463,
  501392,
  500078,
  499590,
  500666,
  499117],

In [10]:
%%time
res = [cpu_task_delayed() for _ in range(100)]
res_computed = compute(res, scheduler='multiprocessing')

Wall time: 1.17 s


In [11]:
io_task_delayed = delayed(io_task)

In [13]:
%%time
res = [io_task_delayed() for _ in range(100)]
res_computed = compute(res, scheduler='threading')

Wall time: 756 ms


In [14]:
%%time
res = [io_task_delayed() for _ in range(100)]
res_computed = compute(res, scheduler='multiprocessing')

Wall time: 1.44 s


## Лабораторная работа 14

1. Напишите функцию, которая считывает файл формата xml из каталога `reviewers_full` и по данным этого файла формирует список словарей, содержащих следующие ключи: `id`, `username`, `name`, `sex`, `country`, `mail`, `registered`, `birthdate`, `name_prefix`, `country_code`. Часть из этих значений в исходном файле хранится в виде тэгов, часть - в виде атрибутов тэгов. Для конкретного человека какие-то из этих ключей могут отсутствовать. 



2. Измерьте время выполнения функции из задания 1 на всех файлах из каталога `reviewers_full`. Ускорьте время выполнения, используя `dask.delayed`.

3. Задекорируйте функцию из задания 1 при помощи `dask.delayed` и создайте список `reviewers`, состоящий из 5 объектов `delayed` (по одному объекту на файл). Из списка объектов `delayed`, создайте `dask.bag` при помощи метода `db.from_delayed`. Добавьте ключ `birth_year`, в котором хранится год рождения человека. Оставьте в выборке только тех людей, которые __наверняка__ моложе 1980 года. Преобразуйте поле `id` к целому типу.

4. Из `dask.bag`, полученного в задании 3, создайте `dask.dataframe` при помощи метода `bag.to_dataframe`. Укажите столбец `id` в качестве индекса.

5. Назовем отзыв негативным, если оценка равна 0, 1 или 2. Загрузите данные о негативных отзывах из файлов архива `reviews_full` (__ЛР12__) в виде `dask.DataFrame`. Посчитайте количество отзывов с группировкой по пользователю, оставившему отзыв. Объедините результат с таблицей, полученной в задаче 4.

#### [версия 2]
* Уточнена формулировка задачи 1